# data-sources

geshun

In [ ]:
reticulate::use_condaenv("base")

## Data Sources

> Answers the question “where is the data and how do I connect to it”.

### Forms of Data

1.  Structured
2.  Semi-structured - json, html, xml
3.  Unstructured

### Data Sources

1.  Flat file
2.  Online Services - AWS, Salesforce, Snowflake
3.  In-house databases
4.  Web - Wikipedia

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3 as sq

In [ ]:
con = sq.connect("data/valuation.sqlite")
cur = con.cursor()

In [ ]:
sql = "select name from sqlite_master where type='table'"
cur.execute(sql)

[('valuation_raw',), ('valuation',), ('valuation_new',)]

In [ ]:
[s for s in dir(pd) if s.find("read") == 0]

['read_clipboard', 'read_csv', 'read_excel', 'read_feather', 'read_fwf', 'read_gbq', 'read_hdf', 'read_html', 'read_json', 'read_orc', 'read_parquet', 'read_pickle', 'read_sas', 'read_spss', 'read_sql', 'read_sql_query', 'read_sql_table', 'read_stata', 'read_table']

In [ ]:
eng = create_engine('sqlite:///data/valuation.sqlite')
pd.read_sql_table("valuation_new", eng)

     supply_date       supplying_company  ... expected_valuation actual_valuation
0     2016-05-18         Delta Logistics  ...              42770            42770
1     2018-08-04              Alpha Inc.  ...              64050            64050
2     2017-12-27              Gamma Ltd.  ...             107230           111070
3     2017-12-01  Beta Supplying Company  ...              99710            99710
4     2016-03-02              Alpha Inc.  ...              79950            79950
...          ...                     ...  ...                ...              ...
1222  2017-11-02              Gamma Ltd.  ...              67310            61860
1223  2020-02-07              Alpha Inc.  ...             112540           112540
1224  2019-08-14  Beta Supplying Company  ...              47210            47210
1225  2016-06-21              Alpha Inc.  ...              57240            57240
1226  2019-05-23              Gamma Ltd.  ...             140920           133670

[1227 rows x 6 

In [ ]:
sql = "select * from valuation_new"
pd.read_sql(sql, con)

              supply_date  ... actual_valuation
0     2016-05-18 00:00:00  ...            42770
1     2018-08-04 00:00:00  ...            64050
2     2017-12-27 00:00:00  ...           111070
3     2017-12-01 00:00:00  ...            99710
4     2016-03-02 00:00:00  ...            79950
...                   ...  ...              ...
1222  2017-11-02 00:00:00  ...            61860
1223  2020-02-07 00:00:00  ...           112540
1224  2019-08-14 00:00:00  ...            47210
1225  2016-06-21 00:00:00  ...            57240
1226  2019-05-23 00:00:00  ...           133670

[1227 rows x 6 columns]

In [ ]:
pd.read_sql(sql, eng)

              supply_date  ... actual_valuation
0     2016-05-18 00:00:00  ...            42770
1     2018-08-04 00:00:00  ...            64050
2     2017-12-27 00:00:00  ...           111070
3     2017-12-01 00:00:00  ...            99710
4     2016-03-02 00:00:00  ...            79950
...                   ...  ...              ...
1222  2017-11-02 00:00:00  ...            61860
1223  2020-02-07 00:00:00  ...           112540
1224  2019-08-14 00:00:00  ...            47210
1225  2016-06-21 00:00:00  ...            57240
1226  2019-05-23 00:00:00  ...           133670

[1227 rows x 6 columns]

In [ ]:
s = """
  select 
    product_code, 
    count(product_code) as n_valuation
  from valuation_new 
  group by product_code
  order by product_code desc
"""
pd.read_sql(s, eng)

  product_code  n_valuation
0          ZUK          143
1          YMD          155
2          WWA          148
3          FOX          146
4          CTC          181
5          BNO          138
6          AMP          151
7          AAD          165

In [ ]:
valuation = pd.read_sql_table("valuation_new", eng)
valuation.product_code.value_counts()

CTC    181
AAD    165
YMD    155
AMP    151
WWA    148
FOX    146
ZUK    143
BNO    138
Name: product_code, dtype: int64

product_code
AAD    165
AMP    151
BNO    138
CTC    181
FOX    146
WWA    148
YMD    155
ZUK    143
dtype: int64

In [ ]:
cur.close()
con.close()